In [1]:
from dotenv import load_dotenv

load_dotenv()

True

**StructuredOutputParser

    - 여러 필드를 반환하고자 할 때 사용할 수 있다.
    - Pydantic/JSON 파서가 더 강력하지만 덜 강력한 모델에 유용하다.

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [6]:
#   `responseSchema` 클래스로 사용자의 질문에 대한 답변과 사용된 소스(웹사이트) 에 대한 설명을 포함하는 응답 스키마를 정의한다.
#   `StructuredOutputParser`를 response_schema를 사용해 초기화해서, 정의된 응답 스미카에 따라 출력을 구조화한다.

response_schema = [
    ResponseSchema(name = 'answer', description='사용자의 질문에 대한 답변'),
    ResponseSchema(
        name='source',
        description='사용자의 질문에 답하기 위해 사용된 출처, 웹사이트여야 함',
    ),
]

# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화

output_parser = StructuredOutputParser.from_response_schemas(response_schema)
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='answer', description='사용자의 질문에 대한 답변', type='string'), ResponseSchema(name='source', description='사용자의 질문에 답하기 위해 사용된 출처, 웹사이트여야 함', type='string')])

    - 응답이 어떻게 포맷되어야 하는지에 대한 지시사항이 담긴 문자열을 받게 되고, 프롬프트에 삽입한다.
    - `output_parser.get_format_instructioin() 으로 함수를 호출해 포맷 지시사항 가져옴`
    - `PromptTemplate`으로 사용자의 질문에 최대한 답변하는 프롬프트 템플릿 생성 (question : 입력변수, format_instructions : 부분 변수)

In [8]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template = "answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=['question'],
    partial_variables={'format_instructions':format_instructions}
)

In [9]:
model = ChatOpenAI(temperature=0,
                   model='gpt-3.5-turbo')

chain = prompt | model | output_parser

In [10]:
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

{'answer': '서울', 'source': 'https://en.wikipedia.org/wiki/Seoul'}

In [11]:
for s in chain.stream({'question' : '세종대왕의 업적은 무엇인가요?'}):
    print(s)

{'answer': '세종대왕은 한글을 창제하고 문화 발전에 큰 기여를 한 대한민국의 역사적 인물입니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%84%B8%EC%A2%85%EB%8C%80%EC%99%95'}
